In [3]:
from src.utils.eval_utils import *

Install seaborn to colorful visualization!


In [1]:
import json 
#pred_data = json.load(open("best_epoch.json", 'r'))
pred_data = json.load(open("/Users/yanjungao/Desktop/results/tuning/LGI_bidir_attn_weightloss_lr4e-3_2020-07-28/predictions/Valid/epoch114.json"))

In [4]:
def load_data(data):
    all_data = {}
    for qid in data['qids']:
        ind = data['qids'].index(qid)
        all_data[qid] = []
        pred = data['predictions'][ind][0]
        gt = data['gts'][ind]
        all_data[qid] = [data['vids'][ind], pred, gt] 
        tiou = compute_tiou(pred, gt)
        all_data[qid].append(tiou)
    return all_data 

all_data = load_data(pred_data)
labels = [] 

In [5]:

def categorize(all_data):
    r3 = []
    r5 = []
    r7 = [] 
    labels = {}
    really_bad = []
    for k,v in all_data.items():
        if v[-1] >= 0.7:
            r7.append(v)
            labels[k] = 'R7'
        elif v[-1] < 0.7 and v[-1] >=0.5:
            r5.append(v)
            labels[k] = 'R5'
        elif v[-1] < 0.5 and v[-1] >= 0.3:
            r3.append(v)
            labels[k] = 'R3'
        else:
            really_bad.append(v)
            labels[k] = 'Else'
    return r3, r5, r7, really_bad, labels


In [6]:
# ActivityNet data 
#pred_data = json.load(open("/Users/yanjungao/Desktop/LGI4temporalgrounding-master/results/anet/predictions/Valid/epoch300.json"))
all_data = load_data(pred_data)
labels = [] 
r3, r5, r7, really_bad, labels = categorize(all_data)
len(labels)

3712

In [7]:
r3, r5, r7, really_bad, labels = categorize(all_data)
len(r7)

1125

In [8]:
cvpr_data = json.load(open("/Users/yanjungao/Desktop/LGI4temporalgrounding-master/pretrained_models/charades_LGI/val_prediction.json"))

In [9]:
cvpr_all_data = load_data(cvpr_data)
cr3, cr5, cr7, creally_bad, cvpr_labels = categorize(cvpr_all_data)

In [10]:
len(cr7)

1314

In [11]:
evaluator = TALLEvaluator()
all_rank1, all_rank5, miou = evaluator.eval(cvpr_data['predictions'],cvpr_data['gts'])

pred: [[4.756972570717335, 11.442847130298615]]	gt: [5.0, 10.0]	tIoU: 0.747845320076292


In [12]:
all_rank1['R1-0.7'] / len(cvpr_data['predictions'])

0.3532258064516129

In [13]:
all_rank1

{'R1-0.1': 2992, 'R1-0.3': 2685, 'R1-0.5': 2201, 'R1-0.7': 1314}

In [14]:
miou / len(cvpr_data['predictions'])

0.5093038932678768

In [15]:
cvpr_all_data

{'0': ['3MSZA',
  [21.106550016403197, 31.57822505950928],
  [24.3, 30.4],
  0.5825238058753427],
 '1': ['3MSZA',
  [22.272344999313354, 31.416261520385742],
  [24.3, 30.4],
  0.6671102022795585],
 '2': ['3MSZA',
  [21.99975398540497, 31.54812355041504],
  [24.3, 30.4],
  0.6388525243465024],
 '3': ['3MSZA',
  [23.513493404388427, 31.52075686454773],
  [24.3, 30.4],
  0.7618083294437571],
 '4': ['AMT7R',
  [3.1837740278244016, 12.721406364440917],
  [4.3, 12.5],
  0.8597521597177605],
 '5': ['YVKIV',
  [0.49458287358284, 9.905966777801515],
  [4.4, 9.2],
  0.5100206355250674],
 '6': ['VXJS4', [0.0, 4.245951200723648], [0.0, 3.4], 0.8007628536617494],
 '7': ['VXJS4', [0.0, 4.037907871305943], [0.0, 4.0], 0.9906120019291865],
 '8': ['GBD1Y',
  [23.879201674461367, 31.117541413307194],
  [26.2, 31.3],
  0.6626701329941153],
 '9': ['GBD1Y',
  [23.525255784988406, 31.697160315513614],
  [26.2, 31.3],
  0.6240895229440749],
 '10': ['KVXJ9', [0.0, 9.905796088278294], [1.0, 11.0], 0.8096178262

In [23]:
all_data['3687']

['RFH6M',
 [14.390306239128114, 20.3004945063591],
 [12.9, 20.2],
 0.7850412909407242]

In [16]:
better = [] 
lines = open("/Users/yanjungao/Desktop/VPMT/data/charades/annotations/charades_sta_test.txt").readlines()

for k,v in all_data.items():
    cvpr_sc = cvpr_all_data[k][-1]
    sc = v[-1]
    query = lines[int(k)].split("##")[-1].strip()
    if sc > cvpr_sc:
        better.append([k, v, query])

In [17]:
lower = [] 
for k,v in all_data.items():
    cvpr_sc = cvpr_all_data[k][-1]
    sc = v[-1]
    query = lines[int(k)].split("##")[-1].strip()
    if sc < cvpr_sc:
        lower.append([k, v, query])
        

In [ ]:
for item in lower:
    print(item)

In [ ]:
len(lower) / len(all_data)

In [ ]:
lines = open("/Users/yanjungao/Desktop/VPMT/data/charades/annotations/charades_sta_test.txt").readlines()
lines[0].split("##")

In [ ]:
len(better) 

In [ ]:
all_data['0']

In [ ]:
for item in better:
    print(item)

In [ ]:
evals = {}
import csv 
with open("analysis_baseline_ours_w_queris_best.csv", "w") as f:
    wr = csv.writer(f)
    for k,v in labels.items():
        query = lines[int(k)].split("##")[-1].strip()
        item = [k,all_data[k][0], v, cvpr_labels[k], query]
        wr.writerow(item)

In [ ]:
labels

In [18]:

def built_matrix(pred, baseline):
    matrix = [[0]*4]*4
    print(matrix)
    for k, v in baseline.items():
        row = v
        if k in pred:
            col = pred[k]
            print("row, col ", row, col)
            matrix[row][col] +=1
    return matrix 
matrix = built_matrix(labels, cvpr_labels)
print(matrix)

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
row, col  R5 R5


TypeError: list indices must be integers or slices, not str

In [19]:
sort_cvpr_label = {}
for k,v in labels.items():
    sort_cvpr_label[k] = cvpr_labels[k]

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=list(labels.values()), y_true=list(sort_cvpr_label.values()),labels=['R3','R5','R7','Else'])

In [21]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "b/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [22]:
print_cm(cm,['R3','R5','R7','Else'])

     b/p     R3    R5    R7  Else 
       R3 134.0 127.0  89.0 132.0 
       R5 162.0 323.0 236.0 164.0 
       R7 129.0 339.0 720.0 123.0 
     Else 118.0 115.0  80.0 721.0 


In [ ]:
(864.0)/ len(labels)